In [1]:
!pip install -q datasets
!pip install -q git+https://github.com/huggingface/transformers.git
!pip install -q bitsandbytes sentencepiece accelerate loralib
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -i https://pypi.org/simple/ bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 24.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 14.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Looking in indexes: https://pypi.org/simple/


In [2]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from PIL import Image
from transformers import IdeficsForVisionText2Text, AutoProcessor, Trainer, TrainingArguments, BitsAndBytesConfig
import torchvision.transforms as transforms

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
checkpoint = "HuggingFaceM4/idefics-9b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_skip_modules=["lm_head", "embed_tokens"],
)
processor = AutoProcessor.from_pretrained(checkpoint)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:

model = IdeficsForVisionText2Text.from_pretrained(checkpoint,quantization_config=bnb_config,device_map="auto")

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.2k [00:00<?, ?B/s]

model-00001-of-00019.safetensors:   0%|          | 0.00/2.00G [00:00<?, ?B/s]

model-00002-of-00019.safetensors:   0%|          | 0.00/1.82G [00:00<?, ?B/s]

model-00003-of-00019.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/1.97G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/705M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [ ]:
import pandas as pd

df=pd.read_csv("/content/ReportWithImage.csv")
# Remove duplicates

df = df.drop_duplicates()
df.to_json('train.jsonl', orient='records', lines=True)

In [ ]:
train_dataset = load_dataset('json', data_files='/content/train.jsonl', split="train")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
data = {
    'User': [],
    'Assistant': []
}
imageUrl=[]

# Map the format and populate the data dictionary
for example in train_dataset:
    user_text = f"User: This is a Picture of Wrist Xray, Is It Fracture or Not in Xray?\nImage: https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/{example['image_name']}?raw=true \n<end_of_utterance>"
    assistant_text = f"\nAssistant: {example['indication']}. Findings: {example['findings']}. Impression: {example['impression']}\nResult: Wrist Xray is {example['Problems']}<end_of_utterance>"
    imageUrl.append(f"https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/{example['image_name']}?raw=true")
    data['User'].append(user_text)
    data['Assistant'].append(assistant_text)

prompts=[]
# Print just the 'text' part
count =0

size = min(len(data['User']), len(data['Assistant']))

# Iterate over the index of each list up to the minimum size
for i in range(size):
    # Create a list containing the values from 'User' and 'Assistant' keys at index i
    if count>=0:
      pair = [data['User'][i], data['Assistant'][i]]
      count =count+1
    else:
       break

    # Append the pair to the prompts list
    prompts.append(pair)


print(prompts)

[['User: This is a Picture of Wrist Xray, Is It Fracture or Not in Xray?\nImage: https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/0015_0668695209_01_WRI-L2_F008.png?raw=true \n<end_of_utterance>', '\nAssistant: There is evidence of torus fracture identified in the distal left radius.\r\nNormal joint spaces\r. Findings: Soft tissue swelling noted.. Impression: Torus fracture of left distal radius. Clinical correlation acquired.\nTo consult Orthopaedic surgeon\n\nResult: Wrist Xray is Fracture<end_of_utterance>'], ['User: This is a Picture of Wrist Xray, Is It Fracture or Not in Xray?\nImage: https://github.com/GoutamSachdev/ReportDataSet/blob/main/Match_fracture/0017_1043285078_01_WRI-L2_F001.png?raw=true \n<end_of_utterance>', '\nAssistant: There is evidence of fractures identified in the left mid shafts of radius ulna.\r\nNormal joint spaces.\r\n. Findings: Soft tissue swelling noted.. Impression: Green stick type fractures of left radius ulna mid shafts. CT for

In [ ]:
len(imageUrl)

522

In [5]:
def do_interence (model,processor, prompt,max_new_tokens=150):
  tokenizer = processor.tokenizer
  bad_words =["<image>","<fake_token_around_image>"]
  if len (bad_words)>0:
    bad_words_ids= tokenizer(bad_words,add_special_tokens=False).input_ids
  eos_token = "</s>"
  eos_token_id=tokenizer.convert_tokens_to_ids(eos_token)

  input=processor(prompt,return_tensors="pt").to(device)
  generated_ids=model.generate(
      **input,
      eos_token_id=[eos_token_id],
      bad_words_ids=bad_words_ids,
      max_new_tokens=max_new_tokens,
      early_stopping=True)
  generated_text=processor.batch_decode(generated_ids,skip_special_tokens=True)[0]
  print( generated_text)
  return generated_text


In [7]:
from peft import get_peft_model, LoraConfig, TaskType, prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj"],
    lora_dropout=0.05,
    bias="none"
)
model = get_peft_model(model, peft_config)



In [65]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token
def format_dataset(examples):
    image_name = examples['image_name']
    prompt = alpaca_prompt.format(examples["instructions"], f"{examples['inputs']}\nhttps://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/{image_name}?raw=true", examples["outputs"]) + EOS_TOKEN


    tokens = tokenizer(prompt,
        truncation=True,
        max_length=256,
        padding="max_length",)
    tokens["labels"] = tokens['input_ids'].copy()
    return tokens


In [56]:
from transformers import AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="right",)
tokenizer.pad_token = tokenizer.eos_token

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [66]:
from datasets import load_dataset

dataset =  load_dataset('csv', data_files='/content/format Report.csv', split="train")
print(dataset[0].keys())
dataset = dataset.map(format_dataset)
print(dataset[0].keys())

dict_keys(['instructions', 'inputs', 'outputs', 'image_name'])


Map:   0%|          | 0/618 [00:00<?, ? examples/s]

dict_keys(['instructions', 'inputs', 'outputs', 'image_name', 'input_ids', 'attention_mask', 'labels'])


In [67]:
print(tokenizer.decode(dataset[0]['input_ids']))

<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Based on the analysis of Wrist Xray, the following issues have been identified:

### Input:
['ronatorsign', 'fracturt', 'toxt']
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/0015_0668695209_01_WRI-L2_F008.png?raw=true

### Response:
There is evidence of torus fracture identified in the distal left radius.
Normal joint spaces
. Soft tissue swelling noted.. Wrist Xray is Fracture</s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s></s>


In [71]:
import torchvision.transforms as transforms

def convert_to_rgb(image):
  if image.mode== "RGB":
    return image
  image_rgba=image.convert("RGBA")
  background = Image.new("RGBA",image_rgba.size,(255,255,255))
  alpha_composite=Image.alpha_composite(background,image_rgba)
  alpha_composite=  alpha_composite.convert("RGB")
  return alpha_composite

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""
EOS_TOKEN = tokenizer.eos_token
def tmp_transforms(examples):
  image_size=processor.image_processor.image_size
  image_mean=processor.image_processor.image_mean
  image_std=processor.image_processor.image_std
  image_transform=transforms.Compose([
      convert_to_rgb,
      transforms.RandomResizedCrop((image_size,image_size),scale=(0.9,1.0),interpolation=transforms.InterpolationMode.BICUBIC),
      transforms.ToTensor(),
      transforms.Normalize(mean=image_mean,std=image_std)
      ])
  prompts=[]
  for i in range(len(examples['instructions'])):
      img=examples['image_name'][0]
      text = alpaca_prompt.format(examples["instructions"], f"""{examples["inputs"]}\n
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/{img}?raw=true""", examples["outputs"]) + EOS_TOKEN # without this token generation goes on forever!
      prompts.append(text)



  inputs=processor(prompts,transform=image_transform,return_tensors="pt").to(device)
  inputs["labels"]=inputs["input_ids"]
  return inputs

In [72]:
tmp = dataset.train_test_split(test_size=0.3)
train_dataset = tmp["train"]
test_dataset = tmp["test"]
train_dataset.set_transform(tmp_transforms)
test_dataset.set_transform(tmp_transforms)
print(train_dataset)
print(tokenizer.decode(train_dataset[0]['input_ids']))
import torch
if torch.cuda.device_count() > 1:
    model.is_parallelizable = True
    model.model_parallel = True

Dataset({
    features: ['instructions', 'inputs', 'outputs', 'image_name', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 432
})
<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
['Based on the analysis of Wrist Xray, the following issues have been identified:']

### Input:
["['text', 'r', 'fracturt', 'softtisos', 'trectune']"]
 
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/5641_0732468047_01_WRI-R1_F011.png?raw=true

### Response:
['There is evidence of torus fracture identified in the right distal radius.. Normal joint spaces.\nSoft tissue swelling noted.\n. Wrist Xray is Fracture']</s>


In [ ]:
tmp

DatasetDict({
    train: Dataset({
        features: ['instructions', 'inputs', 'outputs', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 352
    })
    test: Dataset({
        features: ['instructions', 'inputs', 'outputs', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 151
    })
})

In [73]:
model_name= checkpoint.split("/")[1]
config=LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj","k_proj","v_proj"],
    lora_dropout=0.05,
    bias="none"
)

In [74]:
model= get_peft_model(model,config)
model.gradient_checkpointing_enable()

In [75]:
model.print_trainable_parameters()

trainable params: 19,750,912 || all params: 8,949,430,544 || trainable%: 0.2207


In [76]:
training_args= TrainingArguments(
    output_dir= f"{model_name}-WristXray",
    learning_rate=2e-4,
    fp16=True,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,
    dataloader_pin_memory=False ,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=25,
    save_steps=50,
    max_steps= 50 ,
    logging_steps=10,
    remove_unused_columns=False,
    push_to_hub=False,
    label_names=["input_ids", "labels", "attention_mask"],
    load_best_model_at_end=True,
    report_to="none",
    optim="paged_adamw_8bit"
 )

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1463: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [35]:
from transformers import  DataCollatorForLanguageModeling
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)
trainer=Trainer(
    model=model,
    args=training_args,
    train_dataset =train_dataset,
    eval_dataset=test_dataset,
    tokenizer = tokenizer,
    data_collator = data_collator,

)

max_steps is given, it will override any value given in num_train_epochs


In [36]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:90: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


Step,Training Loss,Validation Loss
25,1.428000,1.421950
50,1.432100,1.421950


TrainOutput(global_step=50, training_loss=1.4233670806884766, metrics={'train_runtime': 1637.4152, 'train_samples_per_second': 0.489, 'train_steps_per_second': 0.031, 'total_flos': 1.2686522648755968e+16, 'train_loss': 1.4233670806884766, 'epoch': 1.8518518518518519})

In [38]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length

In [39]:
max_length = get_max_length(model)
print(max_length)

Found max lenth: 2048
2048


In [40]:
prompt=["<s> Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n['Based on the analysis of Wrist Xray, the following issues have been identified:']\n\n### Input:\n[\"['tent', 'fracture', 'perostealnaction', 'periostealreaction', 'fracture', 'texs']\"]\n \nhttps://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/['0469_1003491236_04_WRI-L1_F009.png']?raw=true\n\n### Response:"]


In [41]:
do_interence(model,processor,prompt)


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:528: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


 Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
['Based on the analysis of Wrist Xray, the following issues have been identified:']

### Input:
["['tent', 'fracture', 'perostealnaction', 'periostealreaction', 'fracture', 'texs']"]
 
https://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/['0469_1003491236_04_WRI-L1_F009.png']?raw=true

### Response:
['The patient has a fracture of the distal end of the radius. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is disp


' Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\n[\'Based on the analysis of Wrist Xray, the following issues have been identified:\']\n\n### Input:\n["[\'tent\', \'fracture\', \'perostealnaction\', \'periostealreaction\', \'fracture\', \'texs\']"]\n \nhttps://github.com/GoutamSachdev/ReportDataSet/blob/main/Images/[\'0469_1003491236_04_WRI-L1_F009.png\']?raw=true\n\n### Response:\n[\'The patient has a fracture of the distal end of the radius. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture is displaced and the fracture is comminuted. The fracture i